# Enedis 13, import, clean, ban formated

In [106]:
# import libraries
import pandas as pd
import numpy as np
import os
# chemin du dataset deja filtré sur le 13
DATA_PATH = 'drive/MyDrive/enedis-wagon/data/'
DATA_CSV = 'enedis_13.csv'
# import du csv pour l'exploration
df = pd.read_csv(os.path.join(DATA_PATH, DATA_CSV))
# Pour générer un dictionnaire de base à modifier
{k : 'float32' for k in df.columns}

{'annee': 'float32',
 'code_iris': 'float32',
 'nom_iris': 'float32',
 'numero_de_voie': 'float32',
 'indice_de_repetition': 'float32',
 'type_de_voie': 'float32',
 'libelle_de_voie': 'float32',
 'code_commune': 'float32',
 'nom_commune': 'float32',
 'segment_de_client': 'float32',
 'nombre_de_logements': 'float32',
 'consommation_annuelle_totale_de_l_adresse_mwh': 'float32',
 'consommation_annuelle_moyenne_par_site_de_l_adresse_mwh': 'float32',
 'consommation_annuelle_moyenne_de_la_commune_mwh': 'float32',
 'adresse': 'float32',
 'tri_des_adresses': 'float32',
 'code_epci': 'float32',
 'code_departement': 'float32',
 'code_region': 'float32'}

In [107]:
# convertir le type des columns
d = {
    'annee': 'int64', # de 2018 a 2022
    #'code_iris': 'str', # inutile?
    #'nom_iris': 'str', # inutile?
    #'numero_de_voie': 'str', # redondance
    #'indice_de_repetition': 'str', # drop indice_de_repetition: 56080 / 57678
    #'type_de_voie': 'str', # redondance
    #'libelle_de_voie': 'str', # redondance
    'code_commune': 'str',
    'nom_commune': 'category',
    #'segment_de_client': 'category', # inutile ils sont tous RESIDENTIEL
    'nombre_de_logements': 'int64',
    'consommation_annuelle_totale_de_l_adresse_mwh': 'float64',
    'consommation_annuelle_moyenne_par_site_de_l_adresse_mwh': 'float64',
    'consommation_annuelle_moyenne_de_la_commune_mwh': 'float64',
    'adresse': 'str',
    #'tri_des_adresses': 'int64', # inutile?
    #'code_epci': 'int64', # inutile?
    #'code_departement': 'int64', # redondance
    #'code_region': 'int64'  # inutile?
 }
df = pd.read_csv(os.path.join(DATA_PATH, DATA_CSV), usecols=d.keys(), dtype=d)

In [108]:
# verification des null
df.isna().sum()

annee                                                      0
code_commune                                               0
nom_commune                                                0
nombre_de_logements                                        0
consommation_annuelle_totale_de_l_adresse_mwh              0
consommation_annuelle_moyenne_par_site_de_l_adresse_mwh    0
consommation_annuelle_moyenne_de_la_commune_mwh            0
adresse                                                    0
dtype: int64

In [109]:
# to_csv du pres clean
df.to_csv(os.path.join(DATA_PATH, 'enedis_13_clean.csv'), index=False)

In [110]:
# load the clean enedis
df_clean = pd.read_csv(os.path.join(DATA_PATH, 'enedis_13_clean.csv'))

In [111]:
# fonction pour l'utilisation de l'api du gouv pour identifier identifiant ban par rapport a l'adresse, nom et code commune
import requests
from io import BytesIO

input_filepath = os.path.join(DATA_PATH, 'enedis_13_clean.csv')
url_csv = 'https://api-adresse.data.gouv.fr/search/csv/'
data = {'result_columns': ['result_id', 'result_score'],
        'columns': ['adresse', 'nom_commune'],
        'citycode': 'code_commune'}

def get_ban_id_from_csv(input_filepath,
                        output_filepath=os.path.join(DATA_PATH, 'enedis_13_clean_ban_formated.csv'),
                        url=url_csv,
                        data=data):

    with open(input_filepath, "rb") as csv_file:
        files = {'data': csv_file}
        r = requests.post(url, files=files, data=data)
        pd.read_csv(BytesIO(r.content)).to_csv(output_filepath, index=False)

In [112]:
# appel de la fonction
%%time
get_ban_id_from_csv(os.path.join(DATA_PATH, 'enedis_13_clean.csv'))

CPU times: user 1.12 s, sys: 106 ms, total: 1.23 s
Wall time: 1min 58s


In [113]:
df_clean_ban_formated = pd.read_csv(os.path.join(DATA_PATH, 'enedis_13_clean_ban_formated.csv'))

In [114]:
# le test du status: resolution impossible
# df_final['result_status'].value_counts()#.isna().sum()
result_status = "Correspondance Identifiant BAN:\nok: 57641\nnot-found: 34\nerror: 3"

In [115]:
# filtre de la fiabilitée de la correspondance
# definis le threshold value de correspondance
threshold = 0.75

# filtrer le DataFrame base sur le threshold
df_clean_ban_formated = df_clean_ban_formated[df_clean_ban_formated['result_score'] >= threshold]
#df_final['result_id'].count()

In [116]:
# Compte rendu
print("Nombre de ligne du dataset original sur le dep13:\n57678\n")
print(result_status)
print(f"\nConcordance avec {threshold} de score:\n{df_clean_ban_formated['result_id'].count()}")
print(f"\nAdresses perdues:\n{57678 - df_clean_ban_formated['result_id'].count()} / 57678")
print(f"\nNombre d'adresse unique:\n{df_clean_ban_formated['result_id'].nunique()}")

Nombre de ligne du dataset original sur le dep13:
57678

Correspondance Identifiant BAN:
ok: 57641
not-found: 34
error: 3

Concordance avec 0.75 de score:
55304

Adresses perdues:
2374 / 57678

Nombre d'adresse unique:
12477


In [117]:
# nettoyage final
# drop column code_commune, nom_commune, adresse et result_score (plus utile, servaient pour recuperer l'identifiant BAN et result score)
co = ['code_commune', 'nom_commune', 'adresse', 'result_score']
df_clean_ban_formated.drop(columns=co, inplace=True)

In [118]:
# renommage des columns
df_clean_ban_formated.rename(columns={'result_id': 'Identifiant__BAN'}, inplace=True)

In [119]:
# export enedis 13 final
df_clean_ban_formated.to_csv(os.path.join(DATA_PATH, 'enedis_13_final.csv'), index=False)

In [120]:
df_clean_ban_formated

,annee,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,Identifiant__BAN
0,2019,16,29.121,1.820,7.149,13085_0241_00004
1,2019,10,69.518,6.952,7.217,13086_0914_00352
2,2019,19,143.778,7.567,7.217,13086_0911_00320
3,2019,22,167.261,7.603,7.322,13076_0025_00594
4,2019,13,51.682,3.976,5.243,13075_3710_00005
...,...,...,...,...,...,...
57673,2021,43,27.335,0.636,4.361,13001_3080_00002
57674,2021,10,15.843,1.584,4.361,13001_3117
57675,2021,42,225.278,5.364,7.077,13002_1770_00009
57676,2021,47,93.977,2.000,7.077,13002_1340_00314


# Merge Enedis with DPE 2022 (csv en local par defaut)

In [121]:
df_clean_ban_formated_2022 = df_clean_ban_formated[df_clean_ban_formated['annee'] == 2022]
df_clean_ban_formated_2022.to_csv(os.path.join(DATA_PATH, 'df_clean_ban_formated_2022.csv'), index=False)

In [122]:
df_clean_ban_formated_2022

,annee,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,Identifiant__BAN
17,2022,14,112.413,8.029,6.720,13030_0290_00016
18,2022,12,44.089,3.674,6.720,13030_uc09nj_00097
19,2022,13,36.555,2.812,6.720,13030_uc09nj_00076
20,2022,11,72.700,6.609,6.076,13088_7920_00035
21,2022,25,45.478,1.819,9.568,13109_0029_00531
...,...,...,...,...,...,...
56828,2022,10,40.366,4.037,6.076,13088_0030_00016
56829,2022,26,59.682,2.295,6.076,13088_7920_00049
56830,2022,10,100.337,10.034,9.568,13109_0020_02250
56831,2022,25,118.526,4.741,9.568,13109_0039_00345


In [123]:
dpe = pd.read_csv(os.path.join(DATA_PATH, 'dpe_groupby_2022.csv'))
dpe

,Identifiant__BAN,DPE_numeric,Etiquette_DPE_per_id_ban
0,13001_0003,4.0,D
1,13001_0003_00013,4.0,D
2,13001_0003_00021,3.0,C
3,13001_0003_00023,3.0,C
4,13001_0003_00025,4.0,D
...,...,...,...
33828,13216_B002,4.0,D
33829,13216_v0pzxz_00007,5.0,E
33830,13216_xkqwwa_00015_b,4.0,D
33831,13216_xkqwwa_00028,3.0,C


In [124]:
df_clean = df_clean_ban_formated_2022.merge(dpe, on='Identifiant__BAN', how='inner')

In [125]:
df_clean.head()

,annee,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,Identifiant__BAN,DPE_numeric,Etiquette_DPE_per_id_ban
0,2022,13,59.079,4.545,5.206,13041_0912_00240,5.0,E
1,2022,49,348.509,7.112,4.470,13005_0484_00105,3.0,C
2,2022,18,63.779,3.543,4.470,13005_1740_00098,6.0,F
3,2022,50,145.142,2.903,4.470,13005_1820_00125,3.0,C
4,2022,11,45.446,4.131,4.470,13005_2025_00130,3.0,C


In [126]:
df_groupby = df_clean.groupby(by='Etiquette_DPE_per_id_ban', sort=True)['consommation_annuelle_moyenne_par_site_de_l_adresse_mwh'].mean().reset_index()

In [127]:
df_groupby.head(10)

,Etiquette_DPE_per_id_ban,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh
0,A,3.838614
1,B,4.129751
2,C,3.709398
3,D,3.034772
4,E,2.898271
5,F,2.947281
6,G,3.254878


In [128]:
df_clean.value_counts('Etiquette_DPE_per_id_ban')

Etiquette_DPE_per_id_ban
D    2499
C    1565
E     676
B     245
F     178
G      49
A      44
Name: count, dtype: int64

In [129]:
import plotly.express as px
import plotly.graph_objects as go
# df_clean['consommation_annuelle_totale_de_l_adresse_mwh'] en y et df_clean['Etiquette_DPE_per_id_ban'] en x et en barplot, sort by df_clean['Etiquette_DPE_per_id_ban']

fig = px.bar(df_groupby, x=df_groupby['Etiquette_DPE_per_id_ban'], y=df_groupby['consommation_annuelle_moyenne_par_site_de_l_adresse_mwh'])
fig.show()



# Resource code (au cas ou)

In [ ]:
# Count the number of rows for each year from 2018 to 2022
year_counts = df_clean_ban_formated["annee"].value_counts().sort_index()

# Filter the year_counts Series to include only the years from 2018 to 2022
year_counts = year_counts.loc[2018:2022]

print(year_counts)

annee
2018    10937
2019    11238
2020    11479
2021    11515
2022    10135
Name: count, dtype: int64


In [ ]:
# selection result au cas ou on doit recuperer de nouvelles columns dans la fonction
'''
df_final[[ 'result_label', 'result_score',
       'result_score_next', 'result_type', 'result_id', 'result_housenumber',
       'result_name', 'result_street', 'result_postcode', 'result_city',
       'result_context', 'result_citycode', 'result_oldcitycode',
       'result_oldcity', 'result_district', 'result_status']].head(3)
'''

"\ndf_final[[ 'result_label', 'result_score',\n       'result_score_next', 'result_type', 'result_id', 'result_housenumber',\n       'result_name', 'result_street', 'result_postcode', 'result_city',\n       'result_context', 'result_citycode', 'result_oldcitycode',\n       'result_oldcity', 'result_district', 'result_status']].head(3)\n"

In [ ]:
# nombre d'adesse unique
#df_final['result_id'].nunique()